#travel insurance 데이터 - 보험가입여부 예측


* travel insurance 데이터를 활용해서 보험가입여부를 예측
* X_train.csv,y_train.csv
* X_test.csv(예측해서 제출하는 것)
* 평가 : roc_auc 평가지표


In [2]:
# 사용할 도구 import
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [3]:
# 데이터 불러오기
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/HyeonSu-Kang/data/refs/heads/main/travel_insurance_train03.csv')
test = pd.read_csv('https://raw.githubusercontent.com/HyeonSu-Kang/data/refs/heads/main/travel_insurance_test03.csv')

In [4]:
Xtrain = train.drop(columns='TravelInsurance')
Xtrain.to_csv('X_train',index= False)
train[['Unnamed: 0','TravelInsurance']].to_csv('y_train',index=False)
test.to_csv('X_test',index=False)

In [5]:
# pd.set_option('display.max_row',500)
# pd.set_option('display.max_columns',20)

In [6]:
# 데이터 불러오기
X = pd.read_csv('X_train')
Y = pd.read_csv('y_train')
X_submission = pd.read_csv('X_test')
display(X.head(2),Y.head(2),X_submission.head(2))

,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,1704,26,Private Sector/Self Employed,Yes,1400000,3,1,No,Yes
1,491,28,Private Sector/Self Employed,Yes,1100000,4,1,No,No


,Unnamed: 0,TravelInsurance
0,1704,0
1,491,1


,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,1569,27,Government Sector,Yes,500000,5,0,No,No
1,1344,25,Private Sector/Self Employed,Yes,1700000,3,0,Yes,No


In [7]:
print(X.info(),X_submission.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
dtypes: int64(5), object(4)
memory usage: 104.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           497 non-null    int64 
 1   Age                  497 non-null    int64 
 2 

In [18]:
# X,X_submission 합쳐 dfX만들고 info()로 결측치, dtype 확인
dfX = pd.concat([X,X_submission],axis = 0)
print(dfX.isna().sum().sum())
dfX.info()
# Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad - object

0
<class 'pandas.core.frame.DataFrame'>
Index: 1987 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1987 non-null   int64 
 1   Age                  1987 non-null   int64 
 2   Employment Type      1987 non-null   object
 3   GraduateOrNot        1987 non-null   object
 4   AnnualIncome         1987 non-null   int64 
 5   FamilyMembers        1987 non-null   int64 
 6   ChronicDiseases      1987 non-null   int64 
 7   FrequentFlyer        1987 non-null   object
 8   EverTravelledAbroad  1987 non-null   object
dtypes: int64(5), object(4)
memory usage: 155.2+ KB


In [9]:
# object 타입으로 된 것들의 Label Encoding 실시
print(dfX['Employment Type'].unique())
dfX['Employment Type'] = dfX['Employment Type'].replace(['Private Sector/Self Employed','Government Sector'],[0,1])

print(dfX['GraduateOrNot'].unique())
dfX['GraduateOrNot'] =  dfX['GraduateOrNot'].replace(['Yes','No'],[0,1])

print(dfX['FrequentFlyer'].unique())
dfX['FrequentFlyer'] = dfX['FrequentFlyer'].replace(['No','Yes'],[0,1])

print(dfX['EverTravelledAbroad'].unique())
dfX['EverTravelledAbroad'] = dfX['EverTravelledAbroad'].replace(['Yes','No'],[0,1])

['Private Sector/Self Employed' 'Government Sector']
['Yes' 'No']
['No' 'Yes']
['Yes' 'No']


<ipython-input-9-3000117666ae>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfX['Employment Type'] = dfX['Employment Type'].replace(['Private Sector/Self Employed','Government Sector'],[0,1])
<ipython-input-9-3000117666ae>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfX['GraduateOrNot'] =  dfX['GraduateOrNot'].replace(['Yes','No'],[0,1])
<ipython-input-9-3000117666ae>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `resu

In [10]:
dfX.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1987 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Unnamed: 0           1987 non-null   int64
 1   Age                  1987 non-null   int64
 2   Employment Type      1987 non-null   int64
 3   GraduateOrNot        1987 non-null   int64
 4   AnnualIncome         1987 non-null   int64
 5   FamilyMembers        1987 non-null   int64
 6   ChronicDiseases      1987 non-null   int64
 7   FrequentFlyer        1987 non-null   int64
 8   EverTravelledAbroad  1987 non-null   int64
dtypes: int64(9)
memory usage: 155.2 KB


In [11]:
# train score,test score roc_auc_score를 반환하는 함수 작성
def get_scores(model,xtrain,xtest,ytrain,ytest):
  A = model.score(xtrain,ytrain)
  B = model.score(xtest,ytest)
  y_pred1 = model.predict_proba(xtrain)[:,1]
  y_pred2 = model.predict_proba(xtest)[:,1]
  C = roc_auc_score(ytrain,y_pred1)
  D = roc_auc_score(ytest,y_pred2)
  return '{:.4f} {:.4f} {:.4f} {:.4f}'.format(A,B,C,D)

In [12]:
# 다양한 모델을 만들고 성능을 출력하는 함수 작성
def make_model(xtrain,xtest,ytrain,ytest):
  model_1 = LogisticRegression().fit(xtrain,ytrain)
  print('model_1',get_scores(model_1,xtrain,xtest,ytrain,ytest))

  for k in range(1,10):
    model_2 = KNeighborsClassifier(k).fit(xtrain,ytrain)
    print('model_2',k,get_scores(model_2,xtrain,xtest,ytrain,ytest))

  model_3 = DecisionTreeClassifier(random_state=0).fit(xtrain,ytrain)
  print('model_3',get_scores(model_3,xtrain,xtest,ytrain,ytest))

  for d in range(3,8):
    model_3 = DecisionTreeClassifier(max_depth=d,random_state=1234).fit(xtrain,ytrain)
    print('model_3',d,get_scores(model_3,xtrain,xtest,ytrain,ytest))

  model_4 = RandomForestClassifier(random_state=1234).fit(xtrain,ytrain)
  print('model_4',get_scores(model_4,xtrain,xtest,ytrain,ytest))

  for i in range(3,8):
    model_4 = RandomForestClassifier(500,random_state=1234,max_depth=i).fit(xtrain,ytrain)
    print('model_4',i,get_scores(model_4,xtrain,xtest,ytrain,ytest))

  model_5 = XGBClassifier().fit(xtrain,ytrain)
  print('model_5',get_scores(model_5,xtrain,xtest,ytrain,ytest))


In [13]:
# X를 train용도,submission 용도로 나누고 Y를 1차원으로 바꿈(MinMaxSclaer적용)
def get_data(dfX,Y):
  X = dfX.drop(columns=['Unnamed: 0'])
  X_use = X.iloc[:1490,:]
  X_submission = X.iloc[1490:,:]
  Y1 = Y['TravelInsurance']
  scaler = MinMaxScaler()
  X1_use = scaler.fit_transform(X_use)
  X1_submission = scaler.transform(X_submission)
  return X1_use,X1_submission,Y1

In [14]:
# 모델 찾기
X1_use,X1_submission,Y1 = get_data(dfX,Y)
xtrain1,xtest1,ytrain1,ytest1 = train_test_split(X1_use,Y1,
                                                 test_size=0.3,
                                                 random_state=1234,
                                                 stratify=Y1)
make_model(xtrain1,xtest1,ytrain1,ytest1)
# model_4 7 0.8504 0.8367 0.9263 0.8153
# model_3 5 0.8418 0.8277 0.8481 0.8364


model_1 0.7699 0.7673 0.7820 0.7695
model_2 1 0.9453 0.7069 0.9394 0.6903
model_2 2 0.8658 0.7629 0.9461 0.7422
model_2 3 0.8610 0.7360 0.9310 0.7566
model_2 4 0.8351 0.7785 0.9133 0.7519
model_2 5 0.8341 0.7785 0.9068 0.7520
model_2 6 0.8284 0.7718 0.8958 0.7568
model_2 7 0.8226 0.7673 0.8909 0.7614
model_2 8 0.8169 0.7673 0.8839 0.7748
model_2 9 0.8073 0.7696 0.8774 0.7694
model_3 0.9501 0.7472 0.9941 0.7260
model_3 3 0.8360 0.8434 0.8173 0.8212
model_3 4 0.8370 0.8389 0.8326 0.8297
model_3 5 0.8418 0.8277 0.8481 0.8364
model_3 6 0.8485 0.8188 0.8651 0.7885
model_3 7 0.8610 0.8076 0.8809 0.7975
model_4 0.9501 0.7740 0.9931 0.7935
model_4 3 0.7987 0.7919 0.8264 0.8074
model_4 4 0.8236 0.8098 0.8419 0.8106
model_4 5 0.8399 0.8367 0.8665 0.8126
model_4 6 0.8428 0.8389 0.8993 0.8140
model_4 7 0.8504 0.8367 0.9263 0.8153
model_5 0.9223 0.7830 0.9813 0.7965


In [15]:
# 최종적으로 사용할 모델 선택
final_model = DecisionTreeClassifier(max_depth=5,random_state=1234).fit(xtrain1,ytrain1)
print('final_model',get_scores(final_model,xtrain1,xtest1,ytrain1,ytest1))

final_model 0.8418 0.8277 0.8481 0.8364


In [16]:
# test 데이터에 대한 확률 구하기.파일로 저장하기
pred = final_model.predict_proba(X1_submission)[:,1]
submission = pd.DataFrame({'ID' : X_submission['Unnamed: 0'],
                           'TravelInsurance' : pred})
submission.to_csv('submission.csv',index=False)

In [17]:
submission.head(10)

,ID,TravelInsurance
0,1569,0.276190
1,1344,1.000000
2,1429,0.182609
3,896,0.294118
4,101,1.000000
5,605,0.182609
6,829,0.182609
7,116,0.182609
8,837,0.182609
9,1466,0.182609
